## Na początek struktury danych których użyłem do stworzenia struktury przypominającej najprostrze AGDS

In [1]:
class Node:
    def __init__(self, hash_key):
        self._hash_key = hash_key

    def __hash__(self):
        return hash(self._hash_key)

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self._hash_key == other._hash_key
        if isinstance(other, str):
            return self._hash_key == other
        raise NotImplementedError(f'__eq__ not implemented for {self}, {other}')

    def __repr__(self):
        return f'Node({self._hash_key})'

### Powyżej klasa opisujaca noda w grafie. W założeniu miała to być klasa bazowa opisująca parametry i atrybuty w grafie czyli to co nie ma "wartości"


In [2]:
class ValueNode(Node):
    def __init__(self, parameter, value, count=1):
        super().__init__(f'{parameter}:{value}')
        self.parameter = parameter
        self.value = value
        self.count = count

    def __repr__(self):
        return f'ValueNode({self.parameter}, {self.value}, {self.count})'

### Dla klasy ValueNode wymyśliłem ,że kluczem pod którym dana wartość będzie przechowywana będzie połączenie nazwy parametru i wartości jaka pod tym parametrem się kryje, dodaje też krotność tej wartości.

### Wstępna klasa tworzaca strukture grafowa jest zaproponowana poniżej 

In [3]:
class AGDS:
    def __init__(self):
        self._graph_dict = {}

    def add_vertex(self, vertex):
        if vertex not in self._graph_dict:
            self._graph_dict[vertex] = []

    def add_edge(self, v1, v2, symmetric=False) -> None:
        connections = [[v1, v2]]

        if symmetric:
            connections += [(v2, v1)]

        for v1, v2 in connections:
            if self._graph_dict[v1]:
                self._graph_dict[v1].append(v2)
            else:
                self._graph_dict[v1] = [v2]

    def add_row(self, row, attributes):
        for param in attributes:
            val = getattr(row, param)

            value_node = ValueNode(param, val)
            if value_node in self._graph_dict:
                self._graph_dict[value_node].count += 1

            else:
                self._graph_dict[value_node] = value_node
                self._graph_dict[param].append(value_node)

    def build_from_pandas(self, pd_dataframe):
        attributes_node = Node('attributes')
        self.add_vertex(attributes_node)

        attrs = [a for a in pd_dataframe.columns]

        for attr in attrs:
            attr_node = Node(attr)
            self.add_vertex(attr_node)
            self.add_edge(attributes_node, attr_node, symmetric=True)

        for row in pd_dataframe.itertuples():
            self.add_row(row, attrs)

### Brakuje jej sortowania wartości w konkretnych parametrach, oraz dodawania połączeń między kolejnymi parametrami. Zdaje sobie z tego sprawę i był to dla mnie zarys struktury jaką powinien mieć ten graf

### W założeniu chciałem do niej dołożyć strukturę ASA pod każdym parametrem, ale nie zdążyłem.

### Budowanie struktury ASA zacząłem od stworzenia swojego algorytmu tworzącego drzewo binarne

In [4]:
class BTreeNode:
    @classmethod
    def split_from_node(cls, node):
        median_key = node.keys[node.t]
        node_left = cls(node.t, leaf=node.leaf)
        node_left.keys = node.keys[:node.t]

        node_right = cls(node.t, leaf=node.leaf)
        node_right.keys = node.keys[node.t + 1:]

        if not node.leaf:
            node_left.children = node.children[:node.t + 1]
            node_right.children = node.children[node.t + 1:]

        return median_key, node_left, node_right

    def __init__(self, t, leaf=False, parent=None):
        self.t = t
        self.keys = []
        self.children = []
        self.leaf = leaf
        self.parent = parent

    def add_key(self, key):
        added = False
        for i in range(len(self.keys)):
            if key < self.keys[i]:
                self.keys.insert(i, key)
                added = True
                break
        if not added:
            self.keys.append(key)

    @property
    def overflow(self):
        return len(self.keys) >= self.t * 2 + 1

### Powyżej jest klasa pomocnicza która opisuje node drzewa.

### Poniżej już właściwa klasa drzewa z metodą insert

In [5]:
class BTree:
    def __init__(self, t):
        self.root = None
        self.t = t

    def insert(self, key):
        if self.root is None:
            self.root = BTreeNode(self.t, True)
            self.root.add_key(key)
            return

        self._insert(key, self.root)

    def _add_and_propagate(self, node, median_key, left_child, right_child):
        parent = node.parent
        if parent is None:
            new_root = BTreeNode(self.t)
            new_root.keys.append(median_key)

            left_child.parent = new_root
            right_child.parent = new_root

            new_root.children.append(left_child)
            new_root.children.append(right_child)

            self.root = new_root
            return

        # search for position in parent node
        for index in range(len(parent.children)):
            if node == parent.children[index]:
                parent.children.pop(index)

                left_child.parent = parent
                right_child.parent = parent

                parent.children.insert(index, left_child)
                parent.children.insert(index + 1, right_child)

        parent.add_key(median_key)

        if parent.overflow:
            median_key_p, left_node_p, right_node_p = BTreeNode.split_from_node(parent)
            self._add_and_propagate(parent, median_key_p, left_node_p, right_node_p)

    def _insert(self, key, node):
        if node.leaf:
            node.add_key(key)
            if node.overflow:
                median_key, left_node, right_node = BTreeNode.split_from_node(node)
                self._add_and_propagate(node, median_key, left_node, right_node)
            return
        else:
            found = False
            for ch_index, n_key in enumerate(node.keys):
                if key < n_key:
                    found = True
                    break

            if not found:
                ch_index += 1

            self._insert(key, node.children[ch_index])

### Poniżej to na czym testowałem czy moja implementacja drzewa zachowuje się sensownie

In [8]:
bt = BTree(1)

bt.insert(2)
bt.insert(9)

bt.insert(1)
bt.insert(4)
bt.insert(5)

bt.insert(3)
bt.insert(6)
bt.insert(10)

### Drzewo nie będzie działało poprawnie dla duplikatów chciałem stworzyć sobie szkic drzewa aby później rozumiejąc podstawy przejść do zaimplementowania struktury ASA i wrzucić te strukture we właściwie miejscie do klasy AGDS dodając kolejne operacje opisane w ASA-GRAPHS FOR EFFICIENT DATA REPRESENTATION AND PROCESSING